# M2 outer control loop axial actuators

This Jupyter notebook is to run the outer control loopouter control loop of M2 axial actuators.
Each axial actuator will be issued with a three forces (20, 50, 100) with pull/push movementall together.

## Import Modules

In [42]:
import asyncio
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [43]:
async def injectForce_axial_actuator(csc, actuators, force, sleep_time=5):
    """injectForce for the axial actuators.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    actuators : list of actuators.
    force : list of forces
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Do the positive direction first
    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0.] * num_axial_actuator
    
    for idx in range(len(actuators)):
        index = actuators[idx]
        forces[index] = force
    await csc.cmd_applyForces.set_start(axial=forces)
    print(f"Apply the force: {abs(force)} N to all axial actuator")   
    await asyncio.sleep(sleep_time)

In [44]:
async def run_ol_axial_test(csc, force, sleep_time=5.0):
    """M2 outer control loop for all actuators.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Print the information of inclinometer
    zenithAngle = csc.tel_zenithAngle.get()
    print(f"Measured zenith angle: {zenithAngle.measured:0.2f} degree.")
    print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
    print(f"Processed inclinometer angle: {zenithAngle.inclinometerProcessed:0.2f} degree.")

    n_act = NUM_ACTUATOR - NUM_TANGENT_LINK
    actuators = [0] * n_act
    for idx in range(n_act):
        actuators[idx] = idx
    
    print(f"Do the  ol test to all actuator: WITH force {force}")
    await injectForce_axial_actuator(csc, actuators, force, sleep_time=sleep_time)

    print("Put to zero all the actuators")
    await m2.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)
    
    print(f"Do the  ol test to all actuator: with force {-force}")
    await injectForce_axial_actuator(csc, actuators, -force, sleep_time=sleep_time)
    
    print("Put to zero all the actuators")
    await m2.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)    

## Prepare the M2 CSC and put to Enabled state

In [45]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [46]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=10)

In [ ]:
"""    
DISABLED = 1
ENABLED = 2
FAULT = 3
OFFLINE = 4
STANDBY = 5

Standby  -->  Disable  -->  Enabled

await m2.cmd_start.set_start(timeout=30)
await m2.cmd_enable.set_start(timeout=200)

3) To Stop, do this:    Enabled  -->  Disable  -->  Standby

await m2.cmd_disable.set_start(timeout=30)
await m2.cmd_standby.set_start(timeout=30) """ 

In [47]:
# get status
state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1685461101.9617786, private_rcvStamp: 1685646348.1978137, private_seqNum: 31, private_identity: MTM2, private_origin: 11350, summaryState: 2


In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=30)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Do the Oouter Loop Test

In [48]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

UTC time to is 2023-06-01 19:05:22.248546 now.


In [49]:
force = 20
await run_ol_axial_test(m2, force, sleep_time=30)

force = 50
await run_ol_axial_test(m2, force, sleep_time=30)

force = 100
await run_ol_axial_test(m2, force, sleep_time=30)

print("Test terminated")

Measured zenith angle: -0.03 degree.
Raw inclinometer angle: 90.03 degree.
Processed inclinometer angle: 89.03 degree.
Do the  ol test to all actuator: WITH force 20
Apply the force: 20 N to all axial actuator
Put to zero all the actuators
Do the  ol test to all actuator: with force -20
Apply the force: 20 N to all axial actuator
Put to zero all the actuators
Measured zenith angle: -0.01 degree.
Raw inclinometer angle: 90.01 degree.
Processed inclinometer angle: 89.05 degree.
Do the  ol test to all actuator: WITH force 50
Apply the force: 50 N to all axial actuator
Put to zero all the actuators
Do the  ol test to all actuator: with force -50
Apply the force: 50 N to all axial actuator
Put to zero all the actuators
Measured zenith angle: -0.01 degree.
Raw inclinometer angle: 90.01 degree.
Processed inclinometer angle: 89.05 degree.
Do the  ol test to all actuator: WITH force 100


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=16965979, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Max axial force (485.39 N) >= 444.82 N.')